# What's cooking?

adaboost

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
train_df = pd.read_json('./train.json')
test_df = pd.read_json('./test.json')

train_df.sample(5)
# coletando ingredientes:
ingredients = train_df.ingredients.str.join(' ')
# ingredients
# removendo caracteres especiais, stemming e lemmatizing:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

train = train_df.copy()
train['X'] = train_df.ingredients.str.join(' ')
train['X'].apply(lemmatize_text)

train.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,cuisine,ingredients,X
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vectorized_ingredients = count_vectorizer.fit_transform(train.X)
print('20 features: '+ str(count_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(count_vectorizer.get_feature_names())))
print(vectorized_ingredients.toarray()[0:10])

vectorized_ingredients_df = pd.DataFrame(columns = count_vectorizer.get_feature_names(), data = vectorized_ingredients.toarray())
vectorized_ingredients_df['num_ingr'] = vectorized_ingredients_df.sum(axis = 1)
vectorized_ingredients_df.head()

20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


,00,10,100,14,15,25,33,40,43,95,...,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices,num_ingr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33


In [3]:
# encode (label) targets:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cuisine = label_encoder.fit_transform(train_df['cuisine'].values)
cuisine

array([ 6, 16,  4, ...,  8,  3, 13])

In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = cuisine.copy()
X = vectorized_ingredients_df.values
num_selected = 1000
X_count_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_count_selected.shape

(39774, 1000)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True)

tfidf_vectorized_ingredients = tfidf_vectorizer.fit_transform(train.X)
print('20 features: '+ str(tfidf_vectorizer.get_feature_names()[0:20]))
print('Total features: '+ str(len(tfidf_vectorizer.get_feature_names())))
print(tfidf_vectorized_ingredients.toarray()[0:10])

tfidf_vectorized_ingredients_df = pd.DataFrame(columns = tfidf_vectorizer.get_feature_names(), data = tfidf_vectorized_ingredients.toarray())
tfidf_vectorized_ingredients_df['sum'] = tfidf_vectorized_ingredients_df.sum(axis = 1)
tfidf_vectorized_ingredients_df.head()


20 features: ['00', '10', '100', '14', '15', '25', '33', '40', '43', '95', '96', 'abalone', 'abbamele', 'absinthe', 'abura', 'acai', 'accent', 'accompaniment', 'achiote', 'acid']
Total features: 3010
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,00,10,100,14,15,25,33,40,43,95,...,za,zatarain,zatarains,zero,zest,zesty,zinfandel,ziti,zucchini,épices
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y = cuisine.copy()
X = tfidf_vectorized_ingredients_df.values.copy()
X_tfidf_selected = SelectKBest(chi2, k=num_selected).fit_transform(X, y)
X_tfidf_selected.shape

(39774, 1000)

In [7]:
# join data
y = cuisine.copy()
print(X_count_selected.shape)
print(X_tfidf_selected.shape)
X = np.concatenate((X_count_selected.copy(),X_tfidf_selected.copy()), axis=1)
# X = tfidf_vectorized_ingredients_df.values.copy()
X.shape

(39774, 1000)
(39774, 1000)


(39774, 2000)

In [8]:
# https://stackoverflow.com/questions/53940258/svc-classifier-taking-too-much-time-for-training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape) #0.85 => 4min #0.6 => 17min
print(y_train.shape)

(26648, 2000)
(26648,)


In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [10]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=100, random_state=42) # 100 = 5min
model.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [11]:
y_pred = model.predict(X_test)
print('Accuracy: '+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred, target_names=train.cuisine.unique()))

Accuracy: 0.5247600182843212
              precision    recall  f1-score   support

       greek       0.34      0.23      0.27       164
 southern_us       0.07      0.00      0.01       271
    filipino       0.50      0.51      0.50       530
      indian       0.53      0.24      0.33       835
    jamaican       0.30      0.25      0.27       225
     spanish       0.32      0.10      0.15       911
     italian       0.63      0.39      0.48       382
     mexican       0.75      0.77      0.76       992
     chinese       0.57      0.23      0.32       222
     british       0.59      0.78      0.68      2581
        thai       0.63      0.43      0.51       150
  vietnamese       0.42      0.16      0.23       488
cajun_creole       0.20      0.72      0.31       275
   brazilian       0.83      0.70      0.76      2138
      french       0.61      0.38      0.47       261
    japanese       0.55      0.28      0.37       155
       irish       0.33      0.63      0.43      142

In [12]:
model.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 100,
 'random_state': 42}

In [13]:
from sklearn.model_selection import GridSearchCV

# parameters = {'learning_rate':[.1, .5, 1, 5, 10, 50], 'n_estimators':[50, 100, 150],'random_state':[42]}
parameters = {'learning_rate':[.5, 1, 10], 'n_estimators':[50, 100, 150],'random_state':[42]}
model2 = AdaBoostClassifier() # 100 = 5min
opt_model = GridSearchCV(
    estimator= model2, 
    param_grid= parameters, 
    verbose = True, 
    cv = 3,
    scoring = 'accuracy',
    return_train_score= True, 
    n_jobs= -1, 
    )

opt_model.fit(X_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3, estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.5, 1, 10],
                         'n_estimators': [50, 100, 150], 'random_state': [42]},
             return_train_score=True, scoring='accuracy', verbose=True)

In [15]:
opt_model.best_params_

{'learning_rate': 0.5, 'n_estimators': 50, 'random_state': 42}

In [16]:
opt_model.best_score_

0.5227031783810095

Mais em:
https://neptune.ai/blog/hyperparameter-tuning-in-python-complete-guide#scikit-optimize